In [1]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

[ RAG 구현 절차 ]

1. 문서의 내용을 읽는다(document_loader를 이용)

- (1) https://python.langchain.com/v0.2/docs/integrations/document_loaders/
- (2) https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
- %pip install --upgrade --quiet docx2txt

2. 문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)

- (1) https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries
- %pip install -qU langchain-text-splitters

3. 쪼갠 문서를 임베딩하여 vector database에 넣음

- (1) OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
- (2) https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
- %pip install –q langchain-chroma

4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)

- (1) https://python.langchain.com/v0.2/docs/tutorials/rag/
- %pip install –q langchain langchainhub

- https://smith.langchain.com/ 에서 key 생성 .env key 추가


In [ ]:
# 문서 읽어오기
# %pip install --upgrade --quiet docx2txt

In [ ]:
# 텍스트를 정크로 나누는 기능만 있는 경량 모듈
# %pip install --upgrade --quiet docx2txt

In [ ]:
# 벡터 데이터베이스 (로컬 데이터베이스)
# %pip install -q langchain-chroma

In [ ]:
# 제공 되는 prompt 사용
# %pip install -q langchain langchainhub

# 1. 문서 읽기 (X)


In [4]:
%%time
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")
document = loader.load()
# documents


CPU times: total: 8.23 s
Wall time: 8.42 s


In [5]:
len(document)

1

In [6]:
document[0].page_content[:200]

'소득세법\n\n소득세법\n\n[시행 2025. 7. 1.] [법률 제20615호, 2024. 12. 31., 일부개정]\n\n기획재정부(재산세제과(양도소득세)) 044-215-4312\n\n기획재정부(소득세제과(근로소득)) 044-215-4216\n\n기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233\n\n기획재정부(소득세제과(사업소득, 기타소득)) 044-2'

# 2. 문서를 쪼개면서 읽기 (O)


In [7]:
import time

start = time.time()

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
)  # 문서를 읽어오면서 겹치게 발라줌

loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")

# 문자단위로 쪼갬
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=200  # 문서를 쪼갤떄 1500글자씩 쪼개
)
# 1번째 chunk 1 ~ 1450글자
# 2번쨰 chunk 1250 ~ 1750글자

document = loader.load_and_split(text_splitter=text_splitter)

runtime = time.time() - start
print("문서 쪼개면서 읽는 시간", runtime)

문서 쪼개면서 읽는 시간 8.540948867797852


In [9]:
# chunk 갯수
len(document)

183

In [10]:
len(document[0].page_content)

1464

In [14]:
# chunk의 글자수들
# [len(doc.page_content) for doc in document]
print(max(len(doc.page_content) for doc in document))
print(min(len(doc.page_content) for doc in document))

1497
1055


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장

- Embedding Model : openAI API text-embedding-3-large (기본 : text-embedding-ada-002)
- 벡터 데이터베이스 : chroma


In [ ]:
# 사용법 : https://python.langchain.com/v0.2/docs/how_to/embed_text/
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
#
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
embeddings = embedding.embed_documents(
    ["소득세법 어쩌구 저쩌구", document[0].page_content]
)

In [21]:
len(embeddings), len(embeddings[0]), len(embeddings[1])

(2, 3072, 3072)

In [ ]:
len(embeddings)

In [8]:
%%time
from langchain_chroma import Chroma

# 데이터를 처음 저잘할떄
# database = Chroma.from_documents(
#     documents=document,
#     embedding=embedding,
#     collection_name='tax_collection',# 생략시 이름 랜덤
#     persist_directory='./chroma',# 생략시 로컬데이터베이스에 저장안됨, 프로그램 종료시 db날라감
# )
# 계속 실행하면 하면 안됨. 꼭 한번만. 그래서 주석처리


CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
%time
load_dotenv()
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

# 이미 저장된 vector DB를 사용할떄
database = Chroma(
    embedding_function=embedding,
    collection_name='tax_collection',
    persist_directory='./chroma'
)

CPU times: total: 0 ns
Wall time: 0 ns


# 4. vector DB 에 질문과 유사도 검색 (답변 생성을 위한 retrieval)


In [ ]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값이 4

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [20]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
[context]는 다음과 같습니다.
{retrieved_docs}
Question: {query}
"""

In [21]:
ai_message = llm.invoke(prompt)

In [22]:
print(ai_message.content)

연봉이 5,000만원인 직장인의 소득세 계산은 다음과 같은 절차를 따릅니다. 여기서 기본적인 공제와 세율을 적용하여 대략적인 세액을 산출하겠습니다.

1. 총급여액(연봉): 5,000만원

2. 근로소득공제 계산:
- 근로소득공제는 소득 금액에 따라 차등 적용됩니다.
- 2025년 세법 기준, 연봉 5,000만원인 경우 근로소득공제는 약 1,910만원 (세법상 계산식에 따라 산출).

3. 과세표준 계산:
- 과세표준 = 총급여액 - 근로소득공제 - 인적공제(본인공제 150만원 기준)
- 과세표준 = 50,000,000 - 19,100,000 - 1,500,000 = 약 29,400,000원

4. 세율 적용:
- 소득세는 누진세율 구간에 따라 계산됩니다.
- 2025년 소득세율 구간에 따라:
  - 과세표준 12,000,000원 이하: 6%
  - 12,000,001 ~ 46,000,000원: 15%

- 세액 계산:
  - 12,000,000원 이하: 12,000,000 * 6% = 720,000원
  - 초과부분 (29,400,000 - 12,000,000): 17,400,000 * 15% = 약 2,610,000원

- 총 세액: 720,000 + 2,610,000 = 약 3,330,000원

5. 근로소득세 감면 및 특례, 지방세(지방소득세 10%)를 고려하지 않은 기본액으로 계산했으며, 실제 납세액은 기타 공제, 세액공제(국민연금, 건강보험, 소득공제 등)를 반영하여 달라질 수 있습니다.

**결론: 약 3,330,000원의 소득세 예상액입니다.**

추가로 상세한 공제 내역이나 세법 변경 내용이 있으면 더 정밀한 계산이 가능하니 참고하시기 바랍니다.


# 5. Agumentation 을 위한 제공하는 Prompt 활용하여, langchain으로 답변 생성

- 사용법 : https://python.langchain.com/v0.2/docs/tutorials/rag/


In [23]:
query = "연봉 5000만원인 직장인의 소득세는 얼마인가요?"

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

## RetrievalQA를 통해 LLM전달 (create_retrieval_chain이 대체)

```
 query -> retriever 전달(벡터 검색 수행)
 -> retriever문서 -> prompt의 {context}에 삽입
 -> query -> prompt의 (question)에 삽입
```


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
ai_message = qa_chain.invoke({"query": query})

In [28]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5000만원인 직장인의 소득세는 정확히 계산하기 어렵지만, 일반적으로 근로소득 공제(최대 2,000만 원 공제 후 과세표준에 대해 세율이 적용)와 기타 공제 등을 고려해야 합니다. 대략적으로, 세법상 기본 세율 구조에 따라 과세표준이 정해지고, 세율은 6%부터 최대 45%까지 적용됩니다. 따라서 최종 세액은 공제 이후 과세표준에 따라 다르며, 구체적으로 계산하려면 상세 소득공제 및 공제액을 반영한 세액계산이 필요합니다.'}